# Programas con estado mutable compartido

En esta sección, presentaremos dos ejercicios de programación implementados en Java, donde cada uno de ellos sufre  del problema anteriormente mencionado: **estado mutable compartido**. La idea en esta sección es entender cómo dichas implementaciones sufren de dicho probleam y, a través de la técnica de copia, vista en un vídeo anterior, eliminar dicho incoveniente.

## Problema 1. (Sistema de transacciones)

Un sistema de transacciones registra las operaciones en moneda extranjera. Supongamos, que tenemos una clase llamada `ForeingEx` que representa divisas en moneda extranjera y su valor de conversión a una moneda determinada, por medio de un valor multiplicativo.

In [ ]:
public class ForeignEx {
   private double value;
   public ForeignEx(double value) {
      this.value = value;
   }
   public double getValue() {
      return value;
   }
   public void setValue(double value) {
      this.value = value;
   }
   public String toString() {
      return "" + value;
   }
}

Implementamos un sencillo sistema de transacciones, por medio de la clase `Transaction`. Esta registra una operación con divisas extranjeras (`ForeignEx`), en ella se registra el monto (`amount`) de cambio.

El siguiente código muestra como se hace una transacción específica.

In [ ]:
public class Transaction {
    private double amount;
    private ForeignEx forEx;
    public Transaction(double amount, ForeignEx forEx) {
       this.amount = amount;
       this.forEx = forEx;
    }
    public double getTransValue() {
       return amount * forEx.getValue();
    }
    public String toString() {
       return "amount: " + " Foreign Exchange: " + forEx + "\nTotal: " + getTransValue() + "\n";
    }
}

El siguiente segmento de código muestra como se llevan a cabo varias transacciones con diferentes montos con una moneda extranjera cuyo valor de conversión es $10.1$ unidades a nuestra moneda local.

In [ ]:
double amounts[] = { 1.2, 3.4, 5.6 };
ForeignEx fe = new ForeignEx(10.1);
List<Transaction> transDay1 = new ArrayList<>();
for (int i = 0; i < amounts.length; i++) {
    transDay1.add(new Transaction(amounts[i],fe));
}
System.out.println(transDay1);

Ahora mostramos otra transacción con un moneda extranjera diferente, que tiene un valor de conversión de $20.1$ unidades a nuestra moneda local.

In [ ]:
double amountsDay2[] = { 2.1, 4.9, 3.2 };
fe = new ForeignEx(20.1);
List<Transaction> transDay2 = new ArrayList<>();
for (int i = 0; i < amountsDay2.length; i++) {
    transDay2.add(new Transaction(amounts[i],fe));
}
System.out.println(transDay2);

Aparentemente todo se encuentra bien. Si imprimimos las transacciones del día uno, se observa que el comportamiento no es el esperado. Esto se debe, a que nuestra implementación de la clase `Transaction` mantiene un estado compartido, este debería guardar una copia, pero guarda una referencia.

In [ ]:
System.out.println(transDay1);

**Reescriba** el código de las transacciones del **cada día** de forma que cada conjunto de transacciones maneje sus divisas extranjeras diferentes en cada transacción.

## Problema 2. (Sistema de registro de versiones)

Este problema se encarga de llevar un registro de las diferentes versiones que puede llegar a tener un fichero determinado.

La clase `Version` se encarga de registrar la versión actual de un fichero (por facilidad con los números de versiones que consta de al menos de tres números en secuencias diferentes X.Y.Z, la vamos a simplicar con único número). Esta clase tiene dos constructores: Uno por omisión que inicia la secuencia en 0 y otro que inicia con un número de versión que iniciar (Se espera que el número de versión sea un número mayor e igual a 0).

In [ ]:
public class Version {
  private int versionNumber;
  public Version(int versionNumber) {
     assert versionNumber >= 0;
     this.versionNumber = versionNumber;
  }
  public Version() {
     this(0);
  }
  public int getVersionNumber() {
     return versionNumber;
  }
  public int incrVersionNumber() {
      return versionNumber++;
  }
  public String toString() {
      return "" + versionNumber;
  }
}

El siguiente segmento de código nos muestra como una versión puede ir cambiando a través del tiempo.

In [ ]:
Version v = new Version();
System.out.println(v);
v.incrVersionNumber();
System.out.println(v);
v.incrVersionNumber();
System.out.println(v);

Utilizando esta clase `Version` vamos a implementar una bitácora de versiones sobre un fichero. Normalmente, la bitácora de versiones lleva el registro de las diferentes versiones que ha pasado por un fichero. Guardaremos nuestro conjunto de versiones por medio de una lista (`List`) de versiones. Para simplificar nuestro problema tendremos un único fichero (archivo). El siguiente código genera la bitácora:

In [ ]:
List<Version> log = new ArrayList<>();
Version fileVersion = new Version();

Vamos a simular el comportamiento de la bitácora, realizando una secuencia de 5 cambios a través del siguiente procedimiento que simula la ejecución de n versiones.

In [ ]:
static void seqChanges(int n) {
    for (int i = 0; i < n; i++) {
        System.out.println("Current version: " + fileVersion.getVersionNumber());
        log.add(fileVersion);
        fileVersion.incrVersionNumber();
        System.out.println("New version: " + fileVersion.getVersionNumber());
    }
}

Probemos que todo (aparentemente) funciona bien:

In [ ]:
seqChanges(5);

Todo bien, hemos hecho los cambios correspondientes y se han actualizado las versiones `[0,1,2,3,4]`. El siguiente valor a registrar es el 5. Entonces, miremos el siguiente código que debe imprimir lo mismo.

In [ ]:
System.out.println(log);

¿Qué acaba de suceder? Lo que acaba de suceder es que el sistema imprime la secuencia de la bitácora como una secuencia de valores: `[5,5,5,5,5]`. Esto es básicamente, un problema del **estado mutable compartido**. El error se presenta en la definición de la clase `Version`. Esta no debe modificar el estado cada vez que incrementa, sino debe debe devolver la siguiente version. Corrija el método `incrVersionNumber` con la nueva firma (**signature**).

```.java
public class Version {
  private int versionNumber;
  public Version(int versionNumber) {
     assert versionNumber >= 0;
     this.versionNumber = versionNumber;
  }
  public Version() {
     this(0);
  }
  public int getVersionNumber() {
     return versionNumber;
  }
  public Version incrVersionNumber() {
      return null; // Retornar la versión nueva
  }
  public String toString() {
      return "" + versionNumber;
  }
}
```

En el siguiente segmento de código se muestra la implementación de una clase auxiliar llamada `Pair`, con ella podemos representar las tuplas en Java, de forma que podamos retornar a través de una función dos valores (`first` y `second`) de cualquier tipo cada uno. 

Igualmente, observas un nuevo código de la clase `Version`, este código **debe ser modificado** con una nueva  implementación del método `incrVersionNumber` de forma que cada vez que se invoca se obtenga una instancia de la clase versión que contiene un número mayor que la clase a la que pertenece.

También, **se debe modificar** el método `seqChanges`. Esta implementará de forma semejanta al cuerpo de código anterior, pero con la premisa, en primer lugar, que recibe: `n` que indica el número de iteraciones y `actual` que es la Versión en la que se parte y retorna a través de la clase auxiliar `Pair` dos valores:  el valor de versión final procesada y una lista con las versiones producidas.

En resumen, se debe implementar en la clase `Version` el método `incrVersionNumber` y el método `seqChange`. Mostrar que la bitácora de realmente lleva el registro de las distinas versiones, eliminando para ello el problema **estado mutable compartido**.

In [ ]:
class Pair<F,S> {
   private F first;
   private S second;
   Pair(F first, S second) {
      this.first = first;
      this.second = second;
   }
   F getFirst() {
      return first;
   }
   S getSecond() {
      return second;
   }
}

public class Version {
  private int versionNumber;
  public Version(int versionNumber) {
     assert versionNumber >= 0;
     this.versionNumber = versionNumber;
  }
  public Version() {
     this(0);
  }
  public int getVersionNumber() {
     return versionNumber;
  }
  public Version incrVersionNumber() {
      return null; // Retornar la versión nueva
  }
  public String toString() {
      return "" + versionNumber;
  }
}

static Pair<Version,List<Version>> seqChange(int n, Version actual) {
    return new null;
}

Version initial = new Version();
Pair<Version,List<Version>> result = seqChange(5, initial);
System.out.println(result.getSecond());